In [2]:
#ACTIVITE MODELES LINEAIRES & FEUILLES D'ARBRES : KNN et SVM
# On importe les librairies dont on aura besoin pour ce tp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn import metrics
%matplotlib inline

# On charge le dataset
house_data = pd.read_csv('Dataset_feuilles_1_train.csv')

#nous aurons besoins de ces 2 librairies pour transformer les données, et générer ytest
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

#On clean le data set.
leaf_data_train = house_data.dropna()
leaf_data_train = leaf_data_train.drop(['id'], axis=1) #.values
#Conversion en dataframe
leaf_data_train = pd.DataFrame(leaf_data_train)
leaf_data_train.head()



,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,0.000000,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,0.000000,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,0.013672,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,0.000000,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [3]:
# On décompose le dataset et on le transforme en matrices pour pouvoir effectuer notre calcul
X = leaf_data_train.iloc[:, 1:] # toutes les colonnes sauf les 2 premieres (indice et species)
Y = leaf_data_train['species'] #colonne species en tant que y

#nous aurons besoins de ces 2 librairies pour centrer réduire les données
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

#l'idée est de faire l'opération sur sur les lignes... et non les labels.
scaler = StandardScaler().fit(X)
X_Scal = scaler.transform(X)





In [4]:
#Vu que le jeu de test ne contient pas de labels, nous allons nous contenter du jeu d'entrainement...
#... et le diviser en jeu d'entrainement et de test (70% train)
print("Vu que le jeu de test ne contient pas de labels, nous allons nous contenter du jeu d'entrainement...")
print("... et le diviser en jeu d'entrainement et de test (70% train)")

#On partage les données : 30% en test
from sklearn import model_selection
xtrain, xtest, ytrain, ytest =  model_selection.train_test_split(X_Scal, Y, test_size=0.3)

Vu que le jeu de test ne contient pas de labels, nous allons nous contenter du jeu d'entrainement...
... et le diviser en jeu d'entrainement et de test (70% train)


In [5]:
#Scénario de base : un classifieur KNN, avec gridsearchCV pour optimiser le nbr de voisins
#d'abord un GridSearchCV pour obtenir le nbr de voisins optimal

#La bibliothèque dont on a besoin
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.grid_search import GridSearchCV
from sklearn import neighbors, metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from statistics import mean


# Fixer les valeurs des hyperparamètres à tester
param_grid = {'n_neighbors':[3, 5, 7, 9, 11, 13, 15, 17, 19, 21]} #, 23, 25, 27, 29, 31, 33, 35, 36, 37, 39, 41]}

# Le score à optimiser, ici l'accuracy
score = 'accuracy'

# Préparer la validation croisée
knn_grid = KNeighborsClassifier()
knn_gridCV = GridSearchCV(knn_grid, param_grid, cv=5)

# On fitte le grid KNN
knn_gridCV.fit(xtrain, ytrain)

#pour obtenir le meilleur voisin
print ("le meilleur nombre de voisins selon GridSearchCV est :", knn_gridCV.best_params_)
print ("la meilleure accuracy selon GridSearchCV est %.4f"% knn_gridCV.best_score_)
GridCV_best_voisins = knn_gridCV.best_params_
GridCV_best_score = knn_gridCV.best_score_

mon_index_maison = knn_gridCV.best_params_.get("n_neighbors")
print ("le nombre de voisins optimal est de", mon_index_maison)

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


le meilleur nombre de voisins selon GridSearchCV est : {'n_neighbors': 3}
la meilleure accuracy selon GridSearchCV est 0.9552
le nombre de voisins optimal est de 3


In [6]:
#On fait donc la prédiction avec le nombre de voisins optimal fourni par gridsearchCV
print("On fait donc la prédiction avec", GridCV_best_voisins," voisins")

# un KNN avec le nombre de voisins donné par GridSearchCV
knn_gridCV_valide = KNeighborsClassifier(n_neighbors= mon_index_maison)
# On créé le modèle avec les données d'entrainement
knn_gridCV_valide.fit(xtrain, ytrain)


# Et on prédit
y_pred_knn = knn_gridCV_valide.predict(xtest)

knn_pred_accuracy = accuracy_score(ytest, y_pred_knn)
print("j'obtiens une accuracy de %.4f"% knn_pred_accuracy, "avec", mon_index_maison, "voisins.")

On fait donc la prédiction avec {'n_neighbors': 3}  voisins
j'obtiens une accuracy de 0.9529 avec 3 voisins.


In [ ]:
#Scénatio alternatif 1 : classifieur SVM sur toutes les données.
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc

  
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['linear', 'sigmoid', 'rbf'], 
             }   


svm = SVC()
#juste pour vérifier la validité de mon dictionnaire param_grid
svm.get_params().keys()

svm_grid = GridSearchCV(svm, param_grid, cv=5) 
svm_grid.fit(xtrain, ytrain) 

#pour obtenir les optimums
print ("les meilleurs paramètres selon GridSearchCV est :", svm_grid.best_params_)
print ("la meilleure accuracy selon GridSearchCV est %.4f"% svm_grid.best_score_)
svm_grid_bestparam = svm_grid.best_params_
svm_grid_bestscore = svm_grid.best_score_

mon_svm_C = svm_grid.best_params_.get('C')
mon_svm_gamma = svm_grid.best_params_.get('gamma')
mon_svm_kernel = svm_grid.best_params_.get('kernel')
print ("ma SVM optimale a un C de", mon_svm_C, ", un gamma de ",mon_svm_gamma, " et un kernel de type ", mon_svm_kernel)

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [9]:
#Scénatio alternatif 1 : classifieur SVM sur toutes les données, prédiction
#On fait donc la prédiction avec les optimums de la SVM
#print("On fait donc la prédiction avec", GridCV_best_voisins," voisins")
print("d'après gridsearchCV, il faut que j'utilise un C de", mon_svm_C, ", un gamma de",mon_svm_gamma, " et un kernel de type", mon_svm_kernel)

# une SVM avec les paramètres donnés par GridSearchCV
svm_pred = SVC(C=mon_svm_C, kernel=mon_svm_kernel, gamma=mon_svm_gamma)
# On créé le modèle avec les données d'entrainement
svm_pred.fit(xtrain, ytrain)


# Et on prédit
y_pred_svm = svm_pred.predict(xtest)

svm_pred_accuracy = accuracy_score(ytest, y_pred_svm)
print("j'obtiens une accuracy de %.4f"% svm_pred_accuracy)


d'après gridsearchCV, il faut que j'utilise un C de 10 , un gamma de 0.001  et un kernel de type rbf
j'obtiens une accuracy de 0.9798


In [10]:
#Scénatio alternatif 2 : classifieur SVM linéaire sur toutes les données.
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc

  
# defining parameter range 
param_grid2 = {'penalty': ['l2'],  
              'C': [0.1, 1, 10, 100, 1000]
              }   


svmlin = LinearSVC()
#juste pour vérifier la validité de mon dictionnaire param_grid
svmlin.get_params().keys()

svmlin_grid = GridSearchCV(svmlin, param_grid2, cv=5) 
svmlin_grid.fit(xtrain, ytrain) 

#pour obtenir les optimums
print ("les meilleurs paramètres selon GridSearchCV est :", svmlin_grid.best_params_)
print ("la meilleure accuracy selon GridSearchCV est %.4f"% svmlin_grid.best_score_)
svmlin_grid_bestparam = svmlin_grid.best_params_
svmlin_grid_bestscore = svmlin_grid.best_score_

mon_svmlin_C = svm_grid.best_params_.get('C')
mon_svmlin_penalty = svm_grid.best_params_.get('penalty')
print ("ma SVM linaire optimale a un C de", mon_svmlin_C, ", et une pénalité de ",mon_svmlin_penalty)

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear fail

les meilleurs paramètres selon GridSearchCV est : {'C': 0.1, 'penalty': 'l2'}
la meilleure accuracy selon GridSearchCV est 0.9481
ma SVM linaire optimale a un C de 10 , et une pénalité de  None


In [11]:
#Scénatio alternatif 2 : classifieur SVM linéaire sur toutes les données.
#On fait donc la prédiction avec les optimums de la SVM linéaire
print("d'après gridsearchCV, il faut que j'utilise un C de", mon_svmlin_C, ", et une pénalité de ",mon_svmlin_penalty)

# une SVM linéaire avec les paramètres donnés par GridSearchCV
svmlin_pred = LinearSVC(C=mon_svmlin_C, penalty= 'l2')
# On créé le modèle avec les données d'entrainement
svmlin_pred.fit(xtrain, ytrain)


# Et on prédit
y_pred_svmlin = svmlin_pred.predict(xtest)

svmlin_pred_accuracy = accuracy_score(ytest, y_pred_svmlin)
print("j'obtiens une accuracy de %.4f"% svmlin_pred_accuracy)

#roc_auc_score(ytest, y_pred_svmlin)

d'après gridsearchCV, il faut que j'utilise un C de 10 , et une pénalité de  None
j'obtiens une accuracy de 0.9461


D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [12]:
#Il est temps de voir ce que le KNN  et la SVM ont donné : résumons les résultats.
print ("Il est temps de voir ce que le KNN  et la SVM ont donné : résumons les résultats.")
print(" ")
print("    Avec le KNN, j'obtiens une accuracy de %.4f"% knn_pred_accuracy, "avec", mon_index_maison, "voisins.")
print("    Avec la SVM, j'obtiens une accuracy de %.4f"% svm_pred_accuracy)
print("    Avec la SVM linéaire, j'obtiens une accuracy de %.4f"% svmlin_pred_accuracy)
print(" ")
print("La SVM est le meilleur modèle pour ce problème.")

Il est temps de voir ce que le KNN  et la SVM ont donné : résumons les résultats.
 
    Avec le KNN, j'obtiens une accuracy de 0.9529 avec 3 voisins.
    Avec la SVM, j'obtiens une accuracy de 0.9798
    Avec la SVM linéaire, j'obtiens une accuracy de 0.9461
 
La SVM est le meilleur modèle pour ce problème.
